In [10]:
import pandas as pd
import numpy as np
import pickle

In [11]:
PROFILE_DIR = "/mnt/storage/research/princeton/resource_estimation/gemm/3090/" 

In [12]:
with open(PROFILE_DIR + "profiling_df.pickle", "rb") as in_file:
    df = pickle.load(in_file)

In [13]:
df

,M,K,N,Kernel Name,Block Size,Grid Size,DRAM Frequency (cycle/second),SM Frequency (cycle/second),Elapsed Cycles (cycle),Memory Throughput (%),...,smsp__pcsamp_warps_issue_stalled_mio_throttle_not_issued (warp),smsp__pcsamp_warps_issue_stalled_misc_not_issued (warp),smsp__pcsamp_warps_issue_stalled_no_instructions_not_issued (inst),smsp__pcsamp_warps_issue_stalled_not_selected_not_issued (warp),smsp__pcsamp_warps_issue_stalled_selected_not_issued (warp),smsp__pcsamp_warps_issue_stalled_short_scoreboard_not_issued (warp),smsp__pcsamp_warps_issue_stalled_sleeping_not_issued (warp),smsp__pcsamp_warps_issue_stalled_tex_throttle_not_issued (warp),smsp__pcsamp_warps_issue_stalled_wait_not_issued (warp),inst_executed (inst)
0,128,128,128,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)","8,435,897,435.90","1,242,788,461.54","8,275",3.11,...,0,0,4,0,0,2,0,0,2,"20,496"
1,128,256,128,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)","8,805,031,446.54","1,292,857,142.86","10,969",4.04,...,0,0,4,0,0,1,0,0,3,"29,888"
2,128,512,128,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 8)","8,465,608,465.61","1,233,985,260.77","7,470",14.78,...,1,0,15,0,0,7,0,0,31,"122,496"
3,128,1024,128,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(2, 1, 11)","8,525,252,525.25","1,245,907,738.10","10,535",15.46,...,2,0,12,0,0,17,0,0,19,"135,008"
4,128,2048,128,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 8)","8,882,352,941.18","1,284,696,691.18","11,195",28.41,...,0,2,15,0,0,29,0,0,21,"249,984"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,65536,1024,16384,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(256, 1, 1)","(512, 64, 1)","9,514,840,282.19","1,397,559,928.33","58,479,339",63.61,...,"46,569","3,177","4,183",0,0,"169,854",0,0,"321,724","3,002,859,520"
910,65536,2048,16384,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(256, 1, 1)","(1024, 32, 1)","9,433,370,856.92","1,385,734,122.92","113,393,829",76.40,...,"25,150","2,191","3,853",0,0,"124,988",0,0,"314,783","5,769,003,008"
911,65536,4096,16384,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(256, 1, 1)","(1024, 32, 1)","9,489,426,717.56","1,394,060,290.47","221,147,610",71.70,...,"16,207","2,261","3,576",0,0,"116,723",0,0,"316,549","11,271,929,856"
912,65536,8192,16384,void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(256, 1, 1)","(1024, 32, 2)","8,960,692,568.17","1,316,390,796.17","478,397,833",82.38,...,"16,999","2,386","3,562",0,0,"116,377",0,0,"316,208","22,623,223,808"
